# 💬 Advanced Sentiment: Aspect-Based & Multilingual

Enterprise-grade sentiment analysis systems.

## Learning Outcomes
- Aspect-based sentiment analysis (ABSA)
- Multilingual sentiment with XLM-RoBERTa
- Emotion detection
- Real-time social media monitoring

**Level**: Advanced | **Time**: 60 min | **GPU**: Recommended

In [ ]:
import torch
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

device = 0 if torch.cuda.is_available() else -1
print(f"Device: {'cuda' if device == 0 else 'cpu'}")

## 1. Basic Sentiment Analysis

In [ ]:
# Load sentiment pipeline
sentiment_pipeline = pipeline('sentiment-analysis', device=device)

texts = [
    "This product is absolutely amazing! Best purchase ever.",
    "Terrible experience. Complete waste of money.",
    "It's okay, nothing special but does the job."
]

print("📊 Basic Sentiment:")
for text in texts:
    result = sentiment_pipeline(text)[0]
    print(f"  {result['label']} ({result['score']:.2%}): {text[:40]}...")

## 2. Aspect-Based Sentiment Analysis

In [ ]:
class AspectSentimentAnalyzer:
    """Analyze sentiment for specific aspects."""
    
    def __init__(self):
        self.classifier = pipeline('sentiment-analysis', device=device)
        self.aspects = ['quality', 'price', 'service', 'delivery', 'design']
    
    def extract_aspects(self, text):
        """Extract mentioned aspects."""
        found = []
        text_lower = text.lower()
        aspect_keywords = {
            'quality': ['quality', 'material', 'build', 'durable'],
            'price': ['price', 'cost', 'expensive', 'cheap', 'value', 'money'],
            'service': ['service', 'support', 'staff', 'customer'],
            'delivery': ['delivery', 'shipping', 'arrived', 'shipped'],
            'design': ['design', 'look', 'beautiful', 'ugly', 'style']
        }
        for aspect, keywords in aspect_keywords.items():
            if any(kw in text_lower for kw in keywords):
                found.append(aspect)
        return found if found else ['overall']
    
    def analyze(self, text):
        """Analyze sentiment per aspect."""
        aspects = self.extract_aspects(text)
        overall = self.classifier(text)[0]
        
        return {
            'text': text,
            'overall_sentiment': overall['label'],
            'overall_score': overall['score'],
            'aspects': aspects
        }

absa = AspectSentimentAnalyzer()

reviews = [
    "Great quality but the price is too high. Delivery was fast!",
    "Terrible customer service. The product design is beautiful though.",
    "Good value for money. Shipping took forever."
]

print("📊 Aspect-Based Sentiment:")
for review in reviews:
    result = absa.analyze(review)
    print(f"\n  Text: {result['text'][:50]}...")
    print(f"  Overall: {result['overall_sentiment']} ({result['overall_score']:.2%})")
    print(f"  Aspects: {result['aspects']}")

## 3. Multilingual Sentiment

In [ ]:
# XLM-RoBERTa for 100+ languages
try:
    multilingual = pipeline(
        'sentiment-analysis',
        model='nlptown/bert-base-multilingual-uncased-sentiment',
        device=device
    )
    
    multilingual_texts = [
        ("English", "This is excellent product!"),
        ("German", "Das ist ein fantastisches Produkt!"),
        ("French", "C'est un produit terrible!"),
        ("Spanish", "¡Este producto es muy bueno!"),
    ]
    
    print("🌍 Multilingual Sentiment:")
    for lang, text in multilingual_texts:
        result = multilingual(text)[0]
        stars = int(result['label'].split()[0])
        print(f"  [{lang}] {'⭐' * stars} ({result['score']:.2%}): {text}")
except:
    print("Install transformers for multilingual support")

## 4. Emotion Detection

In [ ]:
try:
    emotion_classifier = pipeline(
        'text-classification',
        model='j-hartmann/emotion-english-distilroberta-base',
        device=device,
        top_k=None
    )
    
    emotion_texts = [
        "I'm so happy with this purchase!",
        "This is absolutely frustrating!",
        "I'm worried about the quality.",
        "What a pleasant surprise!"
    ]
    
    print("😊 Emotion Detection:")
    for text in emotion_texts:
        result = emotion_classifier(text)[0]
        top_emotion = max(result, key=lambda x: x['score'])
        emoji = {'joy': '😊', 'anger': '😠', 'fear': '😨', 'sadness': '😢', 'surprise': '😲', 'disgust': '🤢', 'neutral': '😐'}
        print(f"  {emoji.get(top_emotion['label'], '❓')} {top_emotion['label']}: {text[:40]}...")
except:
    print("Emotion model requires: pip install transformers")

## 5. Real-Time Social Media Monitor

In [ ]:
class SocialMediaMonitor:
    """Real-time sentiment monitoring system."""
    
    def __init__(self):
        self.sentiment = pipeline('sentiment-analysis', device=device)
        self.history = []
        self.alerts = []
    
    def process_post(self, post, source='unknown'):
        """Process a social media post."""
        result = self.sentiment(post['text'])[0]
        
        entry = {
            'text': post['text'],
            'source': source,
            'sentiment': result['label'],
            'score': result['score'],
            'timestamp': post.get('timestamp', 'now')
        }
        self.history.append(entry)
        
        # Alert on very negative
        if result['label'] == 'NEGATIVE' and result['score'] > 0.9:
            self.alerts.append(entry)
        
        return entry
    
    def get_summary(self):
        """Get sentiment summary."""
        if not self.history:
            return {}
        
        pos = sum(1 for h in self.history if h['sentiment'] == 'POSITIVE')
        neg = sum(1 for h in self.history if h['sentiment'] == 'NEGATIVE')
        
        return {
            'total': len(self.history),
            'positive': pos,
            'negative': neg,
            'positive_rate': pos / len(self.history),
            'alerts': len(self.alerts)
        }

# Demo
monitor = SocialMediaMonitor()

posts = [
    {'text': 'Love your new product! Amazing quality!', 'source': 'twitter'},
    {'text': 'Worst customer service ever!!! Never buying again!!!', 'source': 'twitter'},
    {'text': 'Pretty good overall, would recommend', 'source': 'facebook'},
]

for post in posts:
    monitor.process_post(post, post['source'])

summary = monitor.get_summary()
print("\n📈 Monitoring Summary:")
print(f"  Total posts: {summary['total']}")
print(f"  Positive: {summary['positive']} ({summary['positive_rate']:.0%})")
print(f"  Alerts: {summary['alerts']}")

## 6. Model Comparison

In [ ]:
comparison = pd.DataFrame({
    'Model': ['BERT-base', 'RoBERTa', 'XLM-R', 'DistilBERT', 'twitter-roberta'],
    'Accuracy': ['93%', '95%', '92%', '91%', '94%'],
    'Languages': ['1', '1', '100+', '1', '1'],
    'Speed': ['1x', '0.9x', '0.8x', '2x', '0.9x'],
    'Best For': ['General', 'Accuracy', 'Multilingual', 'Speed', 'Social']
})

print("📊 Sentiment Model Comparison:")
display(comparison)

## 🎯 Key Takeaways
1. ABSA for fine-grained analysis
2. XLM-RoBERTa for multilingual
3. Emotion detection beyond pos/neg
4. Real-time monitoring for brand safety

## 📚 Further Reading
- Pontiki et al., "SemEval ABSA" dataset
- Conneau et al., "XLM-RoBERTa" (2020)
- Mohammad et al., "Emotion lexicons"